In [1]:
import numpy as np
import math
from dataclasses import dataclass
from typing import Callable, Dict, Optional

np.seterr(all="raise")  # Error on overflow

@dataclass
class FoundationParameters:
    REO_DENSITY: float = 7850  # kg/m3
    LOAD_DEAD: float = 8000  # kN
    LOAD_LIVE: float = 2500  # kN
    BEARINGPRESSURE: float = 500  # kPa
    FTG_COVER: float = 0.060  # m
    COLUMN_WIDTH: float = 0.500  # m

@dataclass
class Calculation:
    func: Callable
    short_name: str
    long_name: str
    definition: str = ""
    format_func: Optional[Callable[[float], str]] = None

    def format_value(self, value: float) -> str:
        if self.format_func:
            return self.format_func(value)
        elif isinstance(value, float):
            return f"{value:.3f}"
        else:
            return str(value)

class FoundationCalculator:
    def __init__(self, params: FoundationParameters):
        self.params = params
        self.calculations: Dict[str, Calculation] = {}

    def add_calculation(self, name: str, calc: Calculation):
        self.calculations[name] = calc

    def calculate(self, sizes: np.ndarray) -> np.ndarray:
        results = {name: sizes[name] for name in sizes.dtype.names}
        calculated = set(sizes.dtype.names)
        
        for name, calc in self.calculations.items():
            args = calc.func.__code__.co_varnames
            missing_args = [arg for arg in args if arg not in results]
            
            if not missing_args:
                results[name] = calc.func(**{arg: results[arg] for arg in args})
                calculated.add(name)
            else:
                print(f"Skipping {name}. Missing arguments: {', '.join(missing_args)}")
        
        uncalculated = set(self.calculations.keys()) - calculated
        if len(uncalculated) > 0:
            raise ValueError(f"Unable to calculate all fields. Uncalculated fields: {', '.join(uncalculated)}")

        new_dtype = sizes.dtype.descr + [(name, 'f8') for name in results if name not in sizes.dtype.names]
        new_sizes = np.empty(sizes.shape, dtype=new_dtype)
        for name in sizes.dtype.names:
            new_sizes[name] = sizes[name]
        for name, result in results.items():
            if name not in sizes.dtype.names:
                new_sizes[name] = result
        
        return new_sizes

class FoundationSizer:
    def __init__(self, params: FoundationParameters, calculator: FoundationCalculator):
        self.params = params
        self.calculator = calculator
        self.sizes = self.generate_foundation_sizes()
        self.calculate_all_properties()
        self.column_names = list(self.sizes.dtype.names)

    def generate_foundation_sizes(self) -> np.ndarray:
        FTG_LEN_MIN = (
            math.ceil(
                math.sqrt(
                    (self.params.LOAD_DEAD + self.params.LOAD_LIVE)
                    / self.params.BEARINGPRESSURE
                )
                / 0.05
            )
            * 0.05
        )
        FTG_LEN_MAX = FTG_LEN_MIN + 1
        FTG_LENS = np.round(
            np.arange(FTG_LEN_MIN, FTG_LEN_MAX + 0.001, 0.05, dtype=np.float32), 2
        )

        FTG_DPTH_MIN = (
            math.ceil(
                (
                    4
                    * math.sqrt(3570)
                    * math.sqrt(
                        (4760 * self.params.COLUMN_WIDTH**2) / 3
                        + (1 + (3 * self.params.BEARINGPRESSURE) / 19040)
                        * (self.params.LOAD_DEAD + self.params.LOAD_LIVE)
                    )
                    - (9520 + 3 * self.params.BEARINGPRESSURE)
                    * self.params.COLUMN_WIDTH
                )
                / (19040 + 3 * self.params.BEARINGPRESSURE)
                / 0.05
            )
            * 0.05
        )
        FTG_DPTH_MAX = FTG_DPTH_MIN * 2
        FTG_DPTHS = np.round(
            np.arange(FTG_DPTH_MIN, FTG_DPTH_MAX + 0.001, 0.05, dtype=np.float32), 2
        )

        FTG_CONC_STRENGTHS = np.array([20, 25, 32, 40, 50, 65], dtype=np.float32)

        FTG_REO_SIZES = np.round(
            np.array(
                [0.012, 0.016, 0.02, 0.024, 0.028, 0.032, 0.036, 0.04],
                dtype=np.float32,
            ),
            3,
        )

        FTG_REO_CTS = np.unique(
            np.round(
                np.concatenate(
                    [
                        np.arange(0.1, 0.301, 0.025, dtype=np.float32),
                        np.arange(0.08, 0.301, 0.02, dtype=np.float32),
                    ]
                ),
                3,
            )
        )

        foundation_sizes = np.array(
            np.meshgrid(
                FTG_LENS, FTG_DPTHS, FTG_CONC_STRENGTHS, FTG_REO_SIZES, FTG_REO_CTS
            )
        ).T.reshape(-1, 5)

        return np.rec.array(
            foundation_sizes,
            dtype=[
                ("FtgLength", "f4"),
                ("FtgDepth", "f4"),
                ("fc", "f4"),
                ("ReoSize", "f4"),
                ("ReoCts", "f4"),
            ],
        )

    def calculate_all_properties(self) -> None:
        self.sizes = self.calculator.calculate(self.sizes)

    def remove_fails(self) -> None:
        valid_mask = (
            (self.sizes["Mrat"] <= 1)
            & (self.sizes["VLrat"] <= 1)
            & (self.sizes["VPrat"] <= 1)
            & (self.sizes["Astact"] >= self.sizes["Astreq"])
            & (self.sizes["BPrat"] <= 1)
        )
        self.sizes = self.sizes[valid_mask]

    def filter_match(self, **kwargs) -> None:
        mask = np.ones(len(self.sizes), dtype=bool)
        for key, value in kwargs.items():
            if key in self.column_names:
                mask &= self.sizes[key] == value
        self.sizes = self.sizes[mask]

    def sort_by_cost(self) -> None:
        self.sizes = np.sort(self.sizes, order="Cost")


    def print_foundation_details(self, row_index: int) -> None:
        if row_index < 0 or row_index >= len(self.sizes):
            print(f"Row index {row_index} is out of range.")
            return

        row = self.sizes[row_index]
        max_name_length = max(len(calc.long_name) for calc in self.calculator.calculations.values())
        column_width = max_name_length + 15  # Add some extra space for the short name

        print(f"Details for foundation size at row {row_index}:")
        
        # Print the foundation properties first
        properties = [
            ("Foundation Length","L", "FtgLength", lambda x: f"{x*1000:.0f} mm"),
            ("Foundation Depth", "D", "FtgDepth", lambda x: f"{x*1000:.0f} mm"),
            ("Concrete Strength", "fc","fc", lambda x: f"{x:.0f} MPa"),
            ("Reinforcement Size","Bar", "ReoSize", lambda x: f"N{x*1000:.0f}"),
            ("Reinforcement Spacing", "CTS", "ReoCts", lambda x: f"{x*1000:.0f} mm")
        ]
        
        for long_name, short_name, field_name, format_func in properties:
            value = row[field_name]
            formatted_value = format_func(value)
            print("{:<{width}}  {}".format(f"{long_name} ({short_name}):", formatted_value, width=column_width))
        
        # Print the calculations
        for name, calc in self.calculator.calculations.items():
            value = row[name]
            formatted_value = calc.format_value(value)
            print("{:<{width}}  {}".format(f"{calc.long_name} ({calc.short_name}):", formatted_value, width=column_width))
        
# Usage
params = FoundationParameters()
calculator = FoundationCalculator(params)

# Add calculations
calculator.add_calculation("SWt", Calculation(
    func=lambda FtgDepth, FtgLength: (
        (6 * FtgDepth * FtgLength**2)
    ),
    short_name="SWt",
    long_name="Self Weight of Footing",
    format_func=lambda x: f"{x:.1f} kN"
))

calculator.add_calculation("Pult", Calculation(
    func=lambda SWt: (
        1.2 * (SWt + params.LOAD_DEAD) + 1.5 * params.LOAD_LIVE
    ),
    short_name="P*",
    long_name="Ultimate Load",
    format_func=lambda x: f"{x:.1f} kN"
))

calculator.add_calculation("BPmax", Calculation(
    func=lambda FtgDepth, FtgLength: (
        6 * FtgDepth * FtgLength**2 + params.LOAD_LIVE + params.LOAD_DEAD
    ) / (FtgLength**2),
    short_name="BPm",
    long_name="Maximum Bearing Pressure",
    format_func=lambda x: f"{x:.0f} kPa"
))

calculator.add_calculation("BPrat", Calculation(
    func=lambda BPmax: BPmax / params.BEARINGPRESSURE,
    short_name="BPρ",
    long_name="Bearing Pressure Ratio",
    format_func=lambda x: f"{x*100:.1f}%"
))

calculator.add_calculation("BPult", Calculation(
    func=lambda Pult, FtgLength: Pult / FtgLength**2,
    short_name="BP*",
    long_name="Ultimate Bearing Pressure",
    format_func=lambda x: f"{x:.0f} kPa"
))

calculator.add_calculation("Astact", Calculation(
    func=lambda ReoSize, ReoCts: (250000 * ReoSize**2 * np.pi) / ReoCts,
    short_name="Ast",
    long_name="Actual Steel Area",
    format_func=lambda x: f"{x:.0f} mm²"
))

calculator.add_calculation("Dom", Calculation(
    func=lambda FtgDepth, ReoSize: FtgDepth - params.FTG_COVER - ReoSize / 2,
    short_name="ds",
    long_name="Effective Depth",
    format_func=lambda x: f"{x*1000:.0f} mm"
))


# TO BE IMPLEMENT THE DIFFERENT X & Y CALCS
# def calculate_min_steel_area(FtgDepth, fc, effective_depth):
#     return (228 * FtgDepth**2 * np.sqrt(fc)) / effective_depth

# # Update the original Astmin calculation
# calculator.add_calculation("Astmin_Y", Calculation(
#     func=lambda FtgDepth, fc, Dom_Y: calculate_min_steel_area(FtgDepth, fc, Dom_Y),
#     short_name="Ast min Y",
#     long_name="Minimum Steel Area Y",
#     format_func=lambda x: f"{x:.0f} mm²"
# ))

# # Add the new Astmin_X calculation
# calculator.add_calculation("Astmin_X", Calculation(
#     func=lambda FtgDepth, fc, Dom_X: calculate_min_steel_area(FtgDepth, fc, Dom_X),
#     short_name="Ast min X",
#     long_name="Minimum Steel Area X",
#     format_func=lambda x: f"{x:.0f} mm²"
# ))


calculator.add_calculation("Astmin", Calculation(
    func=lambda FtgDepth, fc, Dom: (228 * FtgDepth**2 * np.sqrt(fc)) / Dom,
    short_name="Ast min",
    long_name="Minimum Steel Area",
    format_func=lambda x: f"{x:.0f} mm²"
))



calculator.add_calculation("alpha", Calculation(
    func=lambda fc: 0.85 - 0.0015 * fc,
    short_name="α",
    long_name="Stress Block Factor",
    format_func=lambda x: f"{x:.3f}"
))

calculator.add_calculation("gamma", Calculation(
    func=lambda fc: 0.97 - 0.0025 * fc,
    short_name="γ",
    long_name="Stress Block Depth Factor",
    format_func=lambda x: f"{x:.3f}"
))

calculator.add_calculation("Mult", Calculation(
    func=lambda FtgLength, BPult, SWt: (
        ((7 * params.COLUMN_WIDTH - 10 * FtgLength)**2 / (8000 * FtgLength**2) * (10 * BPult * FtgLength**2 - 9 * SWt)) 
    ),
    short_name="M*",
    long_name="Ultimate Moment",
    format_func=lambda x: f"{x:.0f} kN·m"
))

calculator.add_calculation("Astshr", Calculation(
    func=lambda Mult, Dom: (5 * Mult) / (2 * Dom),
    short_name="Ass",
    long_name="Steel Area for Shear",
    format_func=lambda x: f"{x:.0f} mm²"
))

calculator.add_calculation("Astreq", Calculation(
    func=lambda Astmin, Astshr: np.maximum(Astmin, Astshr),
    short_name="Astreq",
    long_name="Required Steel Area",
    format_func=lambda x: f"{x:.0f} mm²"
))

calculator.add_calculation("ku", Calculation(
    func=lambda Astreq, alpha, Dom, fc, gamma, FtgLength: Astreq
    / (2000 * alpha * Dom * fc * gamma * FtgLength),
    short_name="ku",
    long_name="Neutral Axis Parameter",
    format_func=lambda x: f"{x:.3f}"
))

calculator.add_calculation("phi", Calculation(
    func=lambda ku: np.minimum(0.85, np.maximum(0.65, 1.24 - 1.08333 * ku)),
    short_name="φ",
    long_name="Capacity Reduction Factor",
    format_func=lambda x: f"{x:.3f}"
))

calculator.add_calculation("fMuo", Calculation(
    func=lambda Astact, Dom, phi, alpha, fc, FtgLength: (Astact * Dom * phi) / 2
    - (Astact**2 * phi) / (8000 * alpha * fc * FtgLength),
    short_name="ØMuo",
    long_name="Moment Capacity",
    format_func=lambda x: f"{x:.0f} kN·m"
))

calculator.add_calculation("Mrat", Calculation(
    func=lambda Mult, fMuo: Mult / fMuo,
    short_name="Mρ",
    long_name="Moment Ratio",
    format_func=lambda x: f"{x*100:.1f}%"
))

calculator.add_calculation("CLR", Calculation(
    func=lambda BPult, Dom: BPult * (params.COLUMN_WIDTH + Dom) ** 2,
    short_name="CLR",
    long_name="Column Load Reaction",
    format_func=lambda x: f"{x:.1f} kN"
))

calculator.add_calculation("VPult", Calculation(
    func=lambda Pult, CLR: Pult - CLR,
    short_name="Vp*",
    long_name="Ultimate Punching Shear",
    format_func=lambda x: f"{x:.1f} kN"
))

calculator.add_calculation("fVP", Calculation(
    func=lambda ReoSize, Dom, fc: (
        952 * (ReoSize - Dom) * (ReoSize - Dom - params.COLUMN_WIDTH) * np.sqrt(fc)
    ),
    short_name="ØVp",
    long_name="Punching Shear Capacity",
    format_func=lambda x: f"{x:.1f} kN"
))

calculator.add_calculation("VPrat", Calculation(
    func=lambda VPult, fVP: VPult / fVP,
    short_name="VPρ",
    long_name="Punching Shear Ratio",
    format_func=lambda x: f"{x*100:.1f}%"
))

calculator.add_calculation("dv", Calculation(
    func=lambda Dom, FtgDepth: np.maximum(0.9 * Dom, 0.72 * FtgDepth),
    short_name="dv",
    long_name="Effective Shear Depth",
    format_func=lambda x: f"{x*1000:.0f} mm"
))

calculator.add_calculation("VLult", Calculation(
    func=lambda BPult, dv, FtgLength: -0.5
    * BPult
    * (params.COLUMN_WIDTH + 2 * dv - FtgLength),
    short_name="VL*",
    long_name="Ultimate Linear Shear",
    format_func=lambda x: f"{x:.1f} kN"
))

calculator.add_calculation("kvo", Calculation(
    func=lambda dv: 2 / (10 + 13 * dv),
    short_name="kvo",
    long_name="Initial Shear Factor",
    format_func=lambda x: f"{x:.3f}"
))

calculator.add_calculation("kv", Calculation(
    func=lambda kvo: np.minimum(kvo, 0.15),
    short_name="kv",
    long_name="Shear Factor",
    format_func=lambda x: f"{x:.3f}"
))

calculator.add_calculation("ks", Calculation(
    func=lambda FtgDepth: np.maximum(0.5, (10 / 7) * (1 - FtgDepth)),
    short_name="ks",
    long_name="Size Factor",
    format_func=lambda x: f"{x:.3f}"
))

calculator.add_calculation("fVuc", Calculation(
    func=lambda dv, fc, ks, kv: 700 * dv * np.sqrt(fc) * ks * kv,
    short_name="ØVuc",
    long_name="Concrete Shear Strength",
    format_func=lambda x: f"{x:.0f} kN"
))

calculator.add_calculation("VLrat", Calculation(
    func=lambda VLult, fVuc: VLult / fVuc,
    short_name="VLρ",
    long_name="Linear Shear Ratio",
    format_func=lambda x: f"{x*100:.1f}%"
))

calculator.add_calculation("Cost", Calculation(
    func=lambda Astact, FtgLength, FtgDepth, fc: (
        Astact / 1000000 * FtgLength * params.REO_DENSITY * 2 * 3400
        + FtgLength**2 * FtgDepth * (130.866 * np.exp(fc * 0.0111) + 45 + 130)
        + 4 * FtgDepth * FtgLength * 180
    ),
    short_name="Cost",
    long_name="Total Cost",
    format_func=lambda x: f"${x:.0f}"
))

# Initialize FoundationSizer and perform calculations
foundation = FoundationSizer(params, calculator)

# Remove fails, sort by cost, and print results
foundation.remove_fails()
foundation.sort_by_cost()
foundation.print_foundation_details(0)

Details for foundation size at row 0:
Foundation Length (L):                    4650 mm
Foundation Depth (D):                     2000 mm
Concrete Strength (fc):                   40 MPa
Reinforcement Size (Bar):                 N24
Reinforcement Spacing (CTS):              150 mm
Self Weight of Footing (SWt):             259.5 kN
Ultimate Load (P*):                       13661.4 kN
Maximum Bearing Pressure (BPm):           498 kPa
Bearing Pressure Ratio (BPρ):             99.5%
Ultimate Bearing Pressure (BP*):          632 kPa
Actual Steel Area (Ast):                  3016 mm²
Effective Depth (ds):                     1928 mm
Minimum Steel Area (Ast min):             2992 mm²
Stress Block Factor (α):                  0.790
Stress Block Depth Factor (γ):            0.870
Ultimate Moment (M*):                     1435 kN·m
Steel Area for Shear (Ass):               1861 mm²
Required Steel Area (Astreq):             2992 mm²
Neutral Axis Parameter (ku):              0.006
Capacity Reducti